In [1]:
import pandas as pd
import json
import numpy as np
from source.transformer import DataTransformer
with open('source/tests/alumnos_test.json', 'r') as archivo_alumnos:
    data=json.loads(archivo_alumnos.read())
dataframe = DataTransformer().transform_to_dataframe(data)
df = dataframe
df.loc[df.nota == 'A', 'nota'] = 7
df = df.loc[df.nota != 'PA'] #No me interesan los pendientes de aprobacion
df['nota'] = df.nota.replace("", np.nan)
df = df.fillna(value=1) #Lleno los austenes con un 1
#df.nota=df.nota.astype(float)
#df['nota'] = df.nota.replace("", np.nan)
#df.nota.fillna(value=1)

/home/nachoyegro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


## ¿Qué materias tenía el alumno X en una fecha determinada?

In [2]:
def materias_hasta(fecha, alumno, df):
    df = df.loc[df.alumno == str(alumno)] #Obtengo los resultados del alumno
    df = df.loc[df.fecha < fecha] #Obtengo las materias hasta la fecha
    return df

materias_hasta('2004-01-01', 3, df)

,materia,nota,alumno,carrera,fecha,resultado
101,00054,6,3,D,2002-12-15,P
102,00055,7,3,D,2003-07-01,P
103,00056,6,3,D,2003-12-15,P
108,00069,7,3,D,2003-07-01,P
109,00070,1,3,D,2003-12-15,U
118,00191,9,3,D,2003-07-01,P
126,00051,7,3,D,2002-12-15,P
129,00273,1,3,D,2003-12-15,U
139,00628,1,3,D,2002-12-15,U
148,00768,6,3,D,2002-12-15,P


## ¿Qué promedio tenía el alumno X en una fecha determinada?

In [3]:
def promedio_hasta(fecha, alumno, df):
    """
        Quiero calcular el promedio del alumno hasta una determinada fecha
        Retorna un numero, que representa el promedio

    """
    df = materias_hasta(fecha, alumno, df) #Obtengo las materias del alumno hasta la fecha
    df.nota=df.nota.astype(float)
    return df.nota.mean()

promedio_hasta('2004-01-01', 1, df)

7.095238095238095

## ¿Qué score tenía el alumno X en una fecha determinada?

In [4]:
def fecha_anterior(fecha):
    """
        Dada una fecha, retorno la misma fecha del año anterior
    """
    from datetime import datetime, timedelta
    return (datetime.strptime(fecha, '%Y-%m-%d') - timedelta(days=365)).strftime('%Y-%m-%d')

def score_hasta(fecha, alumno, df):
    """
        Quiero calcular el score del alumno hasta una determinada fecha
        Se asume que la columna nota del dataframe ya es de tipo float
        El score consiste en el promedio del ultimo año
        Retorna un numero, que representa el promedio

    """
    
    df = materias_hasta(fecha, alumno, df)
    df = df.loc[df.fecha > fecha_anterior(fecha)] #Filtro las materias del ultimo año anterior a "fecha"
    df.nota=df.nota.astype(float)
    return df.nota.mean()

score_hasta('2004-01-01', 1, df)

7.166666666666667

## ¿Qué porcentaje de carrera tenía el alumno X al momento de hacer la materia Y?

Teniendo en cuenta que el numero de materias de una carrera es fijo, ya debería tenerlo en algun lugar, **supongamos que es 40**

In [5]:
def test(row):
    row['porcentaje_carrera'] = 45
    row['promedio'] = promedio_hasta(row.fecha, row.alumno, df)
    row['score'] = score_hasta(row.fecha, row.alumno, df)
    return row
df = df.apply(test, axis=1)
df = df.fillna(value=7) #Lleno los austenes con un 7
df

,materia,nota,alumno,carrera,fecha,resultado,porcentaje_carrera,promedio,score
0,00069,7,1,D,2002-08-22,A,45,7.500000,7.500000
1,00355,7,1,D,2002-08-22,A,45,7.500000,7.500000
2,00834,7,1,D,2002-08-22,A,45,7.500000,7.500000
3,00191,7,1,D,2003-10-31,A,45,7.166667,7.666667
4,E0086,7,1,D,2002-08-22,A,45,7.500000,7.500000
...,...,...,...,...,...,...,...,...,...
149,00777,1,3,D,2011-07-31,U,45,3.705882,1.000000
150,00777,1,3,D,2018-12-20,U,45,3.540000,1.000000
151,90000,1,3,D,2009-01-05,U,45,3.461538,1.000000
152,90000,1,3,D,2011-07-31,U,45,3.705882,1.000000


## Quiero un nuevo DataFrame que me diga, por cada materia qué nota tenia el alumno, qué score tenia y qué porcentaje de carrera tenía al momento de hacer esa materia.

    - Por cada fila, agarro el alumno y la fecha
    - Hago los calculos anteriores y genero una nueva columna por cada dato que quiero agregar. 
    - En este caso, voy a agregar 3 columnas nuevas: Promedio hasta esa fecha, score hasta esa fecha y porcentaje de carrera hasta esa fecha.

# Agrupo datos por semestre

In [6]:
#Dada una fecha, quiero saber a que período pertenece
def fecha_periodo(fecha_str):
    """
        Si la fecha es mayor a octubre:
            periodo: anio-12-31
        Si la fecha es menor a Marzo:
            periodo: anioAnterior-12-31
        Si la fecha es mayor a marzo y menor a octubre
            periodo: anio-06-30
    """
    from datetime import datetime
    fecha = datetime.strptime(str(fecha_str), '%Y-%m-%d')
    if fecha.month > 10:
        return '{}-12-31'.format(fecha.year)
    elif fecha.month <= 3:
        return '{}-12-31'.format(fecha.year - 1)
    else:
        return '{}-06-30'.format(fecha.year)
    return periodo

#fecha_periodo('2019-02-10') == '2018-12-31'

def periodo_semestre(periodo):
    from datetime import datetime
    fecha = datetime.strptime(str(periodo), '%Y-%m-%d')
    if fecha.month == 12:
        return '{}-S2'.format(fecha.year)
    else:
        return '{}-S1'.format(fecha.year)
    return periodo

periodo_semestre('2018-12-31')

'2018-S2'

In [7]:
def periodos(row):
    row['fecha_periodo'] = fecha_periodo(row.fecha)
    row['periodo_semestre'] = periodo_semestre(row['fecha_periodo'])
    return row
df = df.apply(periodos, axis=1)
df = df.fillna(value=7) #Lleno los austenes con un 7
df

,materia,nota,alumno,carrera,fecha,resultado,porcentaje_carrera,promedio,score,fecha_periodo,periodo_semestre
0,00069,7,1,D,2002-08-22,A,45,7.500000,7.500000,2002-06-30,2002-S1
1,00355,7,1,D,2002-08-22,A,45,7.500000,7.500000,2002-06-30,2002-S1
2,00834,7,1,D,2002-08-22,A,45,7.500000,7.500000,2002-06-30,2002-S1
3,00191,7,1,D,2003-10-31,A,45,7.166667,7.666667,2003-06-30,2003-S1
4,E0086,7,1,D,2002-08-22,A,45,7.500000,7.500000,2002-06-30,2002-S1
...,...,...,...,...,...,...,...,...,...,...,...
149,00777,1,3,D,2011-07-31,U,45,3.705882,1.000000,2011-06-30,2011-S1
150,00777,1,3,D,2018-12-20,U,45,3.540000,1.000000,2018-12-31,2018-S2
151,90000,1,3,D,2009-01-05,U,45,3.461538,1.000000,2008-12-31,2008-S2
152,90000,1,3,D,2011-07-31,U,45,3.705882,1.000000,2011-06-30,2011-S1


In [8]:
def materias_alumno_periodo(df, alumno, fecha):
    df = df.loc[df.alumno == str(alumno)] #Obtengo los resultados del alumno
    df = df.loc[df.fecha_periodo == fecha] #Obtengo las materias de una determinada fecha
    return df

def score_alumno_fecha(df, alumno, fecha):
    df = materias_alumno_periodo(df, alumno, fecha)
    df.loc[df.nota == 'A', 'nota'] = 7 #Pongo las aprobadas como si fuese un 7
    df = df.loc[df.nota != 'PA'] #No me interesan los pendientes de aprobacion
    df = df.fillna(value=1) #Lleno los austenes con un 1
    df.nota=df.nota.astype(float)
    return df.nota.mean()

def score_periodo(row):
    row['score_periodo'] = score_alumno_fecha(df, row.alumno, row.fecha_periodo)
    return row

df = df.apply(score_periodo, axis=1)
#score_alumno_fecha(df, 1, '2002-06-30')
#materias_alumno_periodo(df, 1, '2002-06-30')
df

/home/nachoyegro/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,materia,nota,alumno,carrera,fecha,resultado,porcentaje_carrera,promedio,score,fecha_periodo,periodo_semestre,score_periodo
0,00069,7,1,D,2002-08-22,A,45,7.500000,7.500000,2002-06-30,2002-S1,7.066667
1,00355,7,1,D,2002-08-22,A,45,7.500000,7.500000,2002-06-30,2002-S1,7.066667
2,00834,7,1,D,2002-08-22,A,45,7.500000,7.500000,2002-06-30,2002-S1,7.066667
3,00191,7,1,D,2003-10-31,A,45,7.166667,7.666667,2003-06-30,2003-S1,7.500000
4,E0086,7,1,D,2002-08-22,A,45,7.500000,7.500000,2002-06-30,2002-S1,7.066667
...,...,...,...,...,...,...,...,...,...,...,...,...
149,00777,1,3,D,2011-07-31,U,45,3.705882,1.000000,2011-06-30,2011-S1,1.000000
150,00777,1,3,D,2018-12-20,U,45,3.540000,1.000000,2018-12-31,2018-S2,1.000000
151,90000,1,3,D,2009-01-05,U,45,3.461538,1.000000,2008-12-31,2008-S2,2.500000
152,90000,1,3,D,2011-07-31,U,45,3.705882,1.000000,2011-06-30,2011-S1,1.000000


In [9]:
#Primero filtro las materias del alumno
#Despues aplico el score a todas las materias

def scores_periodos(df):
    scores = df[['periodo_semestre', 'score_periodo']].drop_duplicates()
    return scores.sort_values(['periodo_semestre'], ascending=[1])
    
df_alumno = df[df.alumno == '1']
df_alumno = scores_periodos(df_alumno)

#df_alumno.items()
[{"nombre": row["periodo_semestre"], "cantidad": row["score_periodo"]} for index,row in scores_periodos(df_alumno).iterrows()]

[{'nombre': '2002-S1', 'cantidad': 7.066666666666666},
 {'nombre': '2003-S1', 'cantidad': 7.5},
 {'nombre': '2003-S2', 'cantidad': 6.666666666666667},
 {'nombre': '2004-S1', 'cantidad': 7.6},
 {'nombre': '2004-S2', 'cantidad': 6.0},
 {'nombre': '2005-S1', 'cantidad': 6.0},
 {'nombre': '2005-S2', 'cantidad': 6.0},
 {'nombre': '2006-S1', 'cantidad': 8.25},
 {'nombre': '2013-S2', 'cantidad': 9.0}]

In [12]:
pd.Series(df_alumno)

ValueError: Wrong number of items passed 2, placement implies 9